In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip install wandb

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms,models
from torch.utils.data import Dataset,DataLoader
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

import cv2
import os
import random
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import csv
from shutil import copyfile
#import wandb
from copy import deepcopy
from sklearn.manifold import TSNE

In [ ]:
# wandb.login()
# api = wandb.Api()

In [ ]:
# dataset = api.artifact("ankitdipto/CV-in-Retail/GP-Test:v0")
# dataset.download()
#MODEL = api.artifact("ankitdipto/CV-in-Retail/VGG16_c4_c5-hard-ngtv-GAN:v0")
#MODEL.download()

In [ ]:
#!unzip -q /content/drive/MyDrive/datasets/TRAIN-embed.zip -d ./
#!ls /content/artifacts/GP-Test:v0/GP-180/test | wc -l
#!unzip -q "/content/drive/MyDrive/ML_MODELS/extGAN.zip" -d ./
!ln -s /content/drive/MyDrive/datasets/GP-180
!ln -s /content/drive/MyDrive/datasets/Grozi-3.2k-mine/Food
!ln -s /content/drive/MyDrive/datasets/Grozi-3.2k-mine/TRAIN-embed

In [ ]:
!ls /content/TRAIN-embed | wc -l

In [ ]:
TRAIN_DIR = "/content/TRAIN-embed"
TEST_DIR = "/content/GP-180/test"
REF_DIR = "/content/GP-180/train"
#ROOT_DIR = "/content/artifacts/GP-Test:v0/GP-180/train"
#TEST_DIR = "/content/artifacts/GP-Test:v0/GP-180/test"
# SAVE_DIR = "/content/drive/MyDrive/ML_MODELS"
PATH_LABELS = "/content/GP-180/GP-180-labels.csv"
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
GPU = torch.device('cuda',index = 0)
CPU = torch.device('cpu')

In [ ]:
from GAN.models import *

In [ ]:
gen = Generator()
gan_ckpt_dir = "/content/GAN/generator"
checkpoint = tf.train.Checkpoint(generator=gen)
latest = tf.train.latest_checkpoint(gan_ckpt_dir)
checkpoint.restore(latest)

In [ ]:
def load_tf_image(image_file):
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)
  img = tf.cast(image, tf.float32)
  return img

In [ ]:
img = cv2.imread("/content/GP-180/test/s3_112_8.jpg")
print(img.shape)

In [ ]:
img_path = "/content/GP-180/train/1467.jpg"
img = load_tf_image(img_path)

img = preprocess_image(img)
plt.imshow(img)
plt.show()

#anchor_img = tf.convert_to_tensor(img, dtype = tf.float32)
img = load_tf_image(img_path)
#img = cv2.imread(img_path)
#print(img.shape)

img = randomCrop(img,0.7,0.7)

img = tf.convert_to_tensor(img,dtype = tf.float32)
img = preprocess_image(img)
anchor_img = tf.expand_dims(img, axis = 0)
anchor_img = gen(anchor_img,training= True)
anchor_img = tf.squeeze(anchor_img,axis=0).numpy() * 0.5 + 0.5
# print(anchor_img.shape)
print("Output Image",anchor_img.min(),anchor_img.mean(),anchor_img.max())
# print(anchor_img)
plt.imshow(anchor_img)
plt.show()

#print(anchor_img)

In [ ]:
img_list = []
img_list_test = []
img_list_ref = []

for (root,dirs,files) in os.walk(TRAIN_DIR):
    for file in files:
        img_list.append(file)

for (root,dirs,files) in os.walk(TEST_DIR):
    for file in files:
        img_list_test.append(file)

for (root,dirs,files) in os.walk(REF_DIR):
    for file in files:
        img_list_ref.append(file)

print(img_list[:20])
print(img_list_test[:30])
print(img_list_ref[:20])

In [ ]:
!rm -rf ANCHOR_DIR

In [ ]:
ANCHOR_DIR = "/content/drive/MyDrive/datasets/GP-180/anchors"
for img in os.listdir(TRAIN_DIR):
    src_path = os.path.join(TRAIN_DIR,img)
    anchor_path = os.path.join(ANCHOR_DIR,img[:-4] + "(a).jpg")        
    
    anchor_image = load_tf_image(src_path)
    #anchor_image = randomCrop(anchor_image,0.8,0.8)
    anchor_image = preprocess_image(anchor_image)
    anchor_image = tf.expand_dims(anchor_image, axis = 0)
    anchor_image = gen(anchor_image,training= True)
    anchor_image = tf.squeeze(anchor_image,axis=0).numpy() * 0.5 + 0.5   # Converting image[-1,1] to image[0,1]
    print(anchor_image.shape)
    anchor_image = anchor_image * 255.0
    anchor_image = cv2.cvtColor(anchor_image,cv2.COLOR_BGR2RGB)
    #anchor_image.dtype = np.uint8
    print(anchor_path)
    cv2.imwrite(anchor_path,anchor_image)
    #assert False

In [ ]:
img2label = {}
with open(PATH_LABELS) as csv_head:
    data = csv.reader(csv_head,delimiter = ',')
    for row in data:
        img2label[row[0]] = row[1]
print(img2label)

In [ ]:
# img2label = {}
# for img_name in img_list:
#     img2label[img_name] = img_name

In [ ]:
def gauss_blur(img):
    sigma = random.randint(0,4)
    if sigma > 0:
        kernel_size = 6 * sigma + 1
    else:
        kernel_size = 5
    #print(img.shape)
    blur = cv2.GaussianBlur(img,(kernel_size,kernel_size),sigma)
    return blur

def randomCrop(img, crop_ratio_min, crop_ratio_max):
    assert crop_ratio_min <= 1.0 and crop_ratio_min > 0
    assert crop_ratio_max <= 1.0 and crop_ratio_max > 0
    assert crop_ratio_max >= crop_ratio_min
    height, width = np.random.uniform(crop_ratio_min,crop_ratio_max,[2])
    height = int(height * img.shape[0])
    width = int(width * img.shape[1])
    x = np.random.randint(0, img.shape[1] - width)
    y = np.random.randint(0, img.shape[0] - height)
    img = img[y:y+height, x:x+width]
    return img

def change_brightness_saturation(img, bright_f=30, sat_f=0.3): 
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV).astype("float32")
    h, s, v = cv2.split(hsv)
    value = np.random.randint(-bright_f,bright_f+1)
    v = v + value
    v = np.clip(v,0,255)
    sat_f = np.random.uniform(1-sat_f, 1+sat_f)
    s = s * sat_f
    s = np.clip(s,0,255)
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv.astype("uint8"), cv2.COLOR_HSV2RGB)
    return img

def mine_hard_negative(idx):
    ref_embed = embedding_positive[idx]
    bulk_ref_embed = torch.stack([ref_embed for x in range(embedding_positive.shape[0])])

    approx_dist = torch.linalg.norm(bulk_ref_embed - embedding_anchor, dim = 1)
    values, indices = approx_dist.sort()
    return indices[1]     # the 0th item will be identical ,so the hardest negative is the 1st item 

def mine_hard_negative_batch(embedding_matrix):
    hard_negative_batch = []
    #embedding_pos = None
    for ref_embed in embedding_matrix:
        bulk_ref_embed = torch.stack([ref_embed for x in range(embedding_matrix.shape[0])])
        approx_dist = torch.linalg.norm(bulk_ref_embed - embedding_matrix, dim = 1)
        values, indices = approx_dist.sort()
        hard_negative = embedding_matrix[indices[1]]
        hard_negative_batch.append(hard_negative)

    hard_negative_batch = torch.stack(hard_negative_batch,dim = 0)
    return hard_negative_batch

def manipulate(img):
    anchor_image = gauss_blur(img) 
    anchor_image = randomCrop(anchor_image, 0.74, 0.74)
    anchor_image = change_brightness_saturation(anchor_image, bright_f=30, sat_f=0.8)
    anchor_image = cv2.resize(anchor_image, (224, 224))
    
    return anchor_image

def manipulate_batch(imgs):
    anchor_image_batch = []
    for img in imgs:
        anchor_image = gauss_blur(img) 
        anchor_image = randomCrop(anchor_image, 0.74, 0.74)
        anchor_image = change_brightness_saturation(anchor_image, bright_f=30, sat_f=0.8)
        anchor_image = cv2.resize(anchor_image, (224, 224))
        anchor_img_batch.append(anchor_image)
    
    anchor_image_batch = torch.stack(anchor_image,dim = 0)
    return anchor_image_batch

def QualityCheck(emb_pos,emb_anc, sample_size):
    Q = []
    indices = random.sample(range(emb_anc.shape[0]), k = sample_size)
    for idx in indices:
        pos_anc_dist = torch.linalg.norm(emb_pos[idx] - emb_anc[idx], dim =0)
        hard_neg_idx = mine_hard_negative(idx)

        pos_ngtv_dist = torch.linalg.norm(emb_pos[idx] - emb_anc[hard_neg_idx])
        quality = pos_ngtv_dist - pos_anc_dist
        Q.append(quality)
    
    return Q

In [ ]:

class RetailDataset(Dataset):
    def __init__(self,root_dir,img_list,transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.img_list = img_list

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir,self.img_list[idx])
        image = cv2.imread(img_path)
        #image = load_tf_image(img_path)
        if image is None:
            print(img_path,idx,self.img_list[idx])
        image = cv2.resize(image,(224,224))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = torch.from_numpy(image)
        image = image.float() / 255
        image = torch.permute(image,[2,0,1])
        if self.transform:
            image = self.transform(image)
        return image

class AnchorPositiveFetcher(RetailDataset):
    def __init__(self,root_dir,img_list,transform = None):
        super().__init__(root_dir,img_list,transform)
        
    def __getitem__(self, idx_pos):
        positive_image = super().__getitem__(idx_pos)                  # The positive image
        
        anchor_path = os.path.join(self.root_dir,self.img_list[idx_pos])
       
        #-------------- RANDOM manipulation-----------
        
        anchor_image = cv2.imread(anchor_path)
        anchor_image = cv2.cvtColor(anchor_image,cv2.COLOR_BGR2RGB)
        anchor_image = manipulate(anchor_image)
        
        #---------------------------------------------

        anchor_image = torch.from_numpy(anchor_image)
        anchor_image = anchor_image.float() / 255.0
        anchor_image = torch.permute(anchor_image,[2,0,1])
        if self.transform:
            anchor_image = self.transform(anchor_image)                # The processed anchor image

            
        AP_pair = torch.stack([anchor_image,positive_image],dim = 0)
        return AP_pair

"""
class TripletRETAIL(RetailDataset):
    def __init__(self,root_dir,img_list,transform = None):
        super().__init__(root_dir,img_list,transform)

    
        
    def __getitem__(self, idx_pos):
        positive_image = super().__getitem__(idx_pos)                  # The positive image
        
        anchor_path = os.path.join(self.root_dir,self.img_list[idx_pos])
        
        
        
        # ------------ GAN generation ----------------
        # anchor_image = load_tf_image(anchor_path)
        # anchor_image = randomCrop(anchor_image,0.8,0.8)
        # anchor_image = preprocess_image(anchor_image)
        # anchor_image = tf.expand_dims(anchor_image, axis = 0)
        # anchor_image = gen(anchor_image,training= True)
        # anchor_image = tf.squeeze(anchor_image,axis=0).numpy() * 0.5 + 0.5   # Converting image[-1,1] to image[0,1]
        # anchor_image = cv2.resize(anchor_image,(224,224))
        #----------------------------------------------

        #-------------- RANDOM manipulation-----------
        
        anchor_image = cv2.imread(anchor_path)
        anchor_image = cv2.cvtColor(anchor_image,cv2.COLOR_BGR2RGB)
        anchor_image = manipulate(anchor_image)
        
        #---------------------------------------------

        anchor_image = torch.from_numpy(anchor_image)
        anchor_image = anchor_image.float() / 255.0
        anchor_image = torch.permute(anchor_image,[2,0,1])
        if self.transform:
            anchor_image = self.transform(anchor_image)                # The processed anchor image

        no_classes = super().__len__()
        neg_idx = random.randint(0,no_classes - 1)
        negative_image = super().__getitem__(neg_idx)                  # The negative image chosen randomly

        #idx_hard_neg = mine_hard_negative(idx_pos)
        #negative_image = super().__getitem__(idx_hard_neg)

        #print("anchor image mean",anchor_image.mean())        
        triplet = torch.stack([anchor_image,positive_image,negative_image],dim = 0)
        return triplet
"""

In [ ]:
#vgg13 = models.vgg13_bn(pretrained = True)
#vgg16 = models.vgg16_bn(pretrained = True)
#print(vgg16)
#resnet18 = models.resnet18(pretrained = True)
resnet18 = models.resnet18(pretrained = True)
res18 = deepcopy(resnet18)

# Triplet network
We'll train a triplet network, that takes an anchor, positive (same class as anchor) and negative (different class than anchor) examples. The objective is to learn embeddings such that the anchor is closer to the positive example than it is to the negative example by some margin value.

![alt text](images/anchor_negative_positive.png "Source: FaceNet")
Source: [2] *Schroff, Florian, Dmitry Kalenichenko, and James Philbin. [Facenet: A unified embedding for face recognition and clustering.](https://arxiv.org/abs/1503.03832) CVPR 2015.*

**Triplet loss**:   $L_{triplet}(x_a, x_p, x_n) = max(0, m +  \lVert f(x_a)-f(x_p)\rVert_2^2 - \lVert f(x_a)-f(x_n)\rVert_2^2$\)

## Steps
1. Create a dataset returning triplets - **TripletMNIST** class from *datasets.py*, wrapper for MNIST-like classes
2. Define **embedding** *(mapping)* network $f(x)$ - **EmbeddingNet** from *networks.py*
3. Define **triplet** network processing triplets - **TripletNet** wrapping *EmbeddingNet*
4. Train the network with **TripletLoss** - *losses.py*

In [ ]:
class Network13(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_block_3 = vgg13.features[0:21]
        self.conv_block_4 = vgg13.features[21:28]
        self.conv_block_5 = vgg13.features[28:35]
        self.downsample = F.interpolate 

    def forward(self,x):
        x3 = self.conv_block_3(x)
        x4 = self.conv_block_4(x3)
        x5 = self.conv_block_5(x4)

        x4 = self.downsample(x4,size = (7,7))
        x3 = self.downsample(x3,size = (7,7))

        out = torch.cat([x3,x4,x5],dim = 1)
        return out

class Network16(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_block_4 = vgg16.features[0:34]
        self.conv_block_5 = vgg16.features[34:44]
        #self.upsample = nn.Upsample(scale_factor = 2,mode = "nearest") 
        #self.downsample = F.interpolate
        self.maxpool_b4 = nn.MaxPool2d(kernel_size = (14,14))
        self.maxpool_b5 = nn.MaxPool2d(kernel_size = (7,7))

    def forward(self,x):
        x4 = self.conv_block_4(x)
        x5 = self.conv_block_5(x4)

        #x4 = self.downsample(x4,size = (7,7))
        x4 = self.maxpool_b4(x4)
        x5 = self.maxpool_b5(x5)
        #print(x4.shape,x5.shape)

        out = torch.cat([x4,x5],dim = 1)
        return out

class ResNet18_high_reso(nn.Module):
    def __init__(self,FREEZE = False):
        super(ResNet18_high_reso,self).__init__()
        self.entrypoint = nn.Sequential(res18.conv1,
                                        res18.bn1,
                                        res18.relu,
                                        res18.maxpool)
        #self.downConv = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        self.layer1 = res18.layer1
        self.layer2 = res18.layer2
        self.layer3 = res18.layer3
        self.layer4 = res18.layer4
        
        self.layer5 = nn.Conv2d(in_channels = 512,out_channels = 512, kernel_size = 3,stride = 1)

        self.maxpool_b4 = nn.MaxPool2d(kernel_size = (14,14))
        self.maxpool_b5 = nn.MaxPool2d(kernel_size = (7,7))
        
        #self.fc1 = nn.Linear(in_features = 768,out_features = 1024, bias = True)

        if FREEZE:
            self.freeze_backbone()

    def freeze_backbone(self):
        self.entrypoint.requires_grad_(False)
        self.layer1.requires_grad_(False)
        self.layer2.requires_grad_(False)
        self.layer3.requires_grad_(False)
        self.layer4.requires_grad_(False)

    def forward(self,X):
        X0 = self.entrypoint(X)
        X1 = self.layer1(X0)
        X2 = self.layer2(X1)
        X3 = self.layer3(X2)
        X4 = self.layer4(X3)
        X5 = self.layer5(X4)

        #print(X5.shape)
        x4_flat = self.maxpool_b4(X4)
        x5_flat = self.maxpool_b5(X5)
        
        out = torch.cat([x4_flat,x5_flat],dim = 1)
        #print(out.shape)
        #assert False
        out = out.view(X.shape[0],-1)
        return out

class ResNet18(nn.Module):
    def __init__(self,FREEZE = False):
        super(ResNet18,self).__init__()
        self.entrypoint = nn.Sequential(res18.conv1,
                                        res18.bn1,
                                        res18.relu,
                                        res18.maxpool)
        #self.downConv = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        self.layer1 = res18.layer1
        self.layer2 = res18.layer2
        self.layer3 = res18.layer3
        self.layer4 = res18.layer4
        self.maxpool_b3 = nn.MaxPool2d(kernel_size = (14,14))
        self.maxpool_b4 = nn.MaxPool2d(kernel_size = (7,7))
        #self.maxpool_b2 = nn.MaxPool2d(kernel_size = (28,28))
        #self.fc1 = nn.Linear(in_features = 768,out_features = 1024, bias = True)

        if FREEZE:
            self.freeze_backbone()

    def freeze_backbone(self):
        self.entrypoint.requires_grad_(False)
        self.layer1.requires_grad_(False)
        self.layer2.requires_grad_(False)
        self.layer3.requires_grad_(False)
        #self.layer4.requires_grad_(False)

    def forward(self,X):
        X0 = self.entrypoint(X)
        X1 = self.layer1(X0)
        X2 = self.layer2(X1)
        X3 = self.layer3(X2)
        X4 = self.layer4(X3)
        
        x3_flat = self.maxpool_b3(X3)
        x4_flat = self.maxpool_b4(X4)
        
        out = torch.cat([x3_flat,x4_flat],dim = 1)
        out = out.view(X.shape[0],-1)
        #out = self.fc1(out)

        return out

In [ ]:
del model

In [ ]:
#del model
model = ResNet18()
unfrozen_params = [p for p in model.parameters() if p.requires_grad == True]

In [ ]:
input = torch.rand(1,3,448,448)
output = model(input)
print(output.shape)

In [ ]:
model.to(GPU)
pass

In [ ]:
BATCH_SIZE = 124

tsfm = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
# train_ds = TripletRETAIL(TRAIN_DIR,img_list,tsfm)
# train_dl = DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle = False)

train_ds = AnchorPositiveFetcher(TRAIN_DIR,img_list,tsfm)
train_dl = DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle = False)

ref_ds = RetailDataset(REF_DIR,img_list_ref,tsfm)
ref_dl = DataLoader(ref_ds,batch_size = BATCH_SIZE, shuffle = False)

test_ds = RetailDataset(TEST_DIR,img_list_test,tsfm)
test_dl = DataLoader(test_ds, batch_size = BATCH_SIZE * 2, shuffle = False)

stocks_ds = RetailDataset(TRAIN_DIR,img_list,tsfm)
stocks_dl = DataLoader(stocks_ds, batch_size = BATCH_SIZE, shuffle = False)

In [ ]:
MARGIN = 1
loss_fn = nn.TripletMarginLoss(margin = MARGIN)
INIT_LR = 1e-4
EPOCHS = 15
trainable_params = [p for p in model.parameters() if p.requires_grad == True]
optimizer = optim.Adam(trainable_params, lr = INIT_LR)
#scheduler = optim.lr_scheduler.StepLR(optimizer,step_size = 20, gamma = 0.1)
#scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
log_interval = 1
test_interval = 1

In [ ]:
run = wandb.init(project = "CV-in-Retail", name = "vgg16_bn_c4+c5_pool_hdngtv")
#wandb.watch(model,log_freq = 10)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
del embedding_positive
del embedding_anchor

In [ ]:
itr = iter(stocks_dl)
batch = next(itr).to(DEVICE)
output = model(batch)
embed_size = output.view(BATCH_SIZE,-1).shape[1]
#embed_size = 1024
embedding_positive = torch.randn(len(train_ds),embed_size).to(GPU)
embedding_anchor = torch.randn(len(train_ds),embed_size).to(GPU)

In [ ]:
#model.to(GPU)
print(embed_size)

In [ ]:
best_ckpt = {"model_state" : model.state_dict(),
            "optimizer_state" : optimizer.state_dict(),
            "epochs" : EPOCHS,
            "margin" : MARGIN,
            "init_lr" : INIT_LR,
            "loss" : float("inf")}

In [ ]:
def train(model,start_epoch,stop_epoch):
    global best_ckpt
    torch.cuda.empty_cache()
    model.train()
    for epoch in range(start_epoch,stop_epoch):
        print("epoch ",epoch)
        cost = 0
        avg_dist = 0
        for (i,batch_AP) in enumerate(tqdm(train_dl)):
            #print(f"Epoch {epoch},batch {i}")
            idx = i * BATCH_SIZE
            #triple = torch.permute(triple,[1,0,2,3,4])
            #triple = triple.to(DEVICE)
            
            curr_batch_size = batch_AP.shape[0]
            batch_AP = batch_AP.to(GPU)
            optimizer.zero_grad()

            batch_AP = torch.permute(batch_AP,[1,0,2,3,4])
            positive = batch_AP[0]
            anchor = batch_AP[1]
            
            positive_emb = model(positive).view(curr_batch_size,-1)
            anchor_emb = model(anchor).view(curr_batch_size,-1)

            negative_emb = mine_hard_negative_batch(positive_emb)

            #anchor, positive, negative = triple[0],triple[1] ,triple[2]
            
            
            #negative = model(negative).view(curr_batch_size,-1)

            positive_emb = positive_emb / torch.linalg.norm(positive_emb,ord=2,dim =1,keepdim = True)
            negative_emb = negative_emb / torch.linalg.norm(negative_emb, dim = 1 ,ord=2,keepdim = True)     # The vectors must be normalised to learn richer representations
            anchor_emb = anchor_emb / torch.linalg.norm(anchor_emb, dim = 1,ord =2, keepdim = True) 

            loss = loss_fn(anchor_emb,positive_emb,negative_emb)
            loss.backward()
            cost += loss.item()
        
            embedding_anchor[idx:idx + curr_batch_size] = anchor_emb.to(GPU)
            embedding_positive[idx :idx + curr_batch_size] = positive_emb.to(GPU)
            optimizer.step()
        
        # scheduler.step()
        if best_ckpt["loss"] > cost:
            best_ckpt = {"model_state" : model.state_dict(),
                "optimizer_state" : optimizer.state_dict(),
                "epochs" : epoch,
                "margin" : MARGIN,
                "init_lr" : INIT_LR,
                "loss" : cost}
        
        if epoch%log_interval == 0:
                            
            Q = QualityCheck(embedding_positive,embedding_anchor,300)
            Q_avg = sum(Q)/len(Q)
            
            #_,acc1,acc5,_ = test(model,20)
            #wandb.log({"Train Loss": cost})
            #wandb.log({"Embedding Quality":Q_avg})
            # wandb.log({"Val Accuracy top 1":acc1})
            # wandb.log({"Val Accuracy top 5":acc5})

            print(f"\nLoss at epoch {epoch} = {cost}, Embedding quality = {Q_avg}")
        
        if epoch > 0 and epoch % test_interval == 0:
            #pass
            print("Starting evaluation ....")
            ref_embeddings = extract_embeddings(ref_dl,model,embed_size)
            test_embeddings = extract_embeddings(test_dl,model,embed_size)
            preds ,acc1, acc5,acc10, acc20 = test(model,ref_embeddings,test_embeddings)
            print("The accuracies are:\n Top 1 accuracy = {0}\n Top 5 accuracy = {1}\n Top 10 accuracy = {2}\n Top 20 accuracy = {3}".format(acc1,acc5,acc10,acc20))


In [ ]:
#test_interval = 1
train(model,0,7)

In [ ]:
Q = QualityCheck(embedding_positive,embedding_anchor,900)
Q_avg = sum(Q)/len(Q)
print(Q_avg.item())

In [ ]:
cost = 0
for (i,batch) in enumerate(tqdm(stocks_dl)):
    #print(f"batch {i}")
    idx = i * BATCH_SIZE
    #triple = torch.permute(triple,[1,0,2,3,4])
    batch = batch.to(DEVICE)
    
    optimizer.zero_grad()

    anchor, positive, negative = batch,batch,batch
    curr_batch_size = anchor.shape[0]
    # print(curr_batch_size)
    positive = model(positive).view(curr_batch_size,-1)
    #anchor = positive
    #negative = positive
    anchor = model(anchor).view(curr_batch_size,-1)
    negative = model(negative).view(curr_batch_size,-1)

    positive = positive / torch.linalg.norm(positive,ord=2,dim =1,keepdim = True)
    negative = negative / torch.linalg.norm(negative, dim = 1 ,ord=2,keepdim = True)     # The vectors must be normalised to learn richer representations
    anchor = anchor / torch.linalg.norm(anchor, dim = 1,ord =2, keepdim = True) 

    loss = loss_fn(anchor,positive,negative)
    loss.backward()
    #cost += loss.item()

    embedding_anchor[idx:idx + curr_batch_size] = anchor.to(GPU)
    embedding_pos[idx :idx + curr_batch_size] = positive.to(GPU)
    optimizer.step()


In [ ]:
PATH = os.path.join(SAVE_DIR,"VGG16-triplet:v2.pt")

In [ ]:
print(best_ckpt["loss"],best_ckpt["epochs"])

In [ ]:
print(best_ckpt)

In [ ]:
last_ckpt = deepcopy(best_ckpt)
last_ckpt["model_state"] = model.state_dict()
last_ckpt["loss"] = 5.378
last_ckpt["epochs"] = 252
last_ckpt["acc_top_1"] = 0.241
last_ckpt["acc_top_5"] = 0.433
last_ckpt["acc_top_10"] = 0.508

In [ ]:
#model2 = Network16()
model.load_state_dict(best_ckpt["model_state"])

In [ ]:
best_ckpt["epochs"] = 96
best_ckpt["acc_top_1"] = 0.37
best_ckpt["acc_top_5"] = 0.61
best_ckpt["acc_top_10"] = 0.7

In [ ]:
artifact = wandb.Artifact("VGG16_c4_c5-dn-hdngt-proper",type = "model")

In [ ]:
# last_ckpt = {"Note" : "A vgg16 trained on actual dataset without learning rate scheduler",
#             "model_state" : model.state_dict(),
#             "optimizer_state" : optimizer.state_dict(),
#             "epochs" : EPOCHS,
#             "margin" : MARGIN,
#             "init_lr" : INIT_LR,
#             "loss" : 287.6784973144531}

torch.save(last_ckpt,"/content/VGG16-C4-C5-dn-hdngtv-ep252.pt")
artifact.add_file("/content/VGG16-C4-C5-dn-hdngtv-ep252.pt")
run.log_artifact(artifact)

In [ ]:
!ls /content/GP-embed-74/train | grep -i Food_Tea_54

In [ ]:
def distance(t1,t2):
    del_t = t1 - t2
    return torch.linalg.vector_norm(del_t)

In [ ]:
val_ds = RetailDataset(ROOT_DIR,img_list,tsfm)
val_dl = DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle = False)

In [ ]:
sample = torch.permute(val_ds[0],[1,2,0]) * 250
sample = torch.clamp(sample,min = 0,max = 255)
plt.imshow(sample)

In [ ]:
torch.cuda.empty_cache()
model2 = Network16()
ckpt = torch.load("/content/artifacts/VGG16_c4_c5-hard-ngtv-GAN:v0/VGG16-C4-C5-hard-neg-gan.pt", map_location = DEVICE)
model2.load_state_dict(ckpt["model_state"])

In [ ]:
ckpt = torch.load("/content/artifacts/VGG13_c3_c4_c5-hard-ngtv:v0/VGG13-triplet-hard-neg.pt", map_location = DEVICE)
model.load_state_dict(ckpt["model_state"])

In [ ]:
#model2 = Network16()
model2.to(GPU)
pass

In [ ]:
def plot_embeddings(embeddings, targets, xlim=None, ylim=None):
    plt.figure(figsize=(10,10))
    for i in range(10):
        inds = np.where(targets==i)[0]
        plt.scatter(embeddings[inds,0], embeddings[inds,1], alpha=0.5, color=colors[i])
    if xlim:
        plt.xlim(xlim[0], xlim[1])
    if ylim:
        plt.ylim(ylim[0], ylim[1])
    plt.legend(mnist_classes)

def extract_embeddings(dataloader, model, D):

    embedder_dim = D

    with torch.no_grad():
        model.eval()
        embeddings = torch.zeros((len(dataloader.dataset), embedder_dim)).to(GPU)
    
        k = 0
        for (nb,batch) in enumerate(tqdm(dataloader)):
            batch = batch.to(DEVICE)
            batch_size = batch.shape[0]
            output = model(batch).view(batch_size,-1)
            output = output / torch.linalg.norm(output,ord=2,dim =1,keepdim = True)
            embeddings[k : k + batch_size] = output
            k += batch_size

    return embeddings

In [ ]:
modelf = ResNet18()
modelf.load_state_dict(best_ckpt["model_state"])

In [ ]:
_ = modelf.to(GPU)

In [ ]:
ref_embeddings = extract_embeddings(ref_dl,modelf,embed_size)
test_embeddings = extract_embeddings(test_dl,modelf,embed_size)

In [ ]:
len(ref_ds)

In [ ]:
#!rm /content/TRAIN-embed/food_2668.jpg

img_list = []
img_list_test = []

for (root,dirs,files) in os.walk(TRAIN_DIR):
    for file in files:
        img_list.append(file)

for (root,dirs,files) in os.walk(TEST_DIR):
    for file in files:
        img_list_test.append(file)
print(img_list[:12])
print(img_list_test[:10])

BATCH_SIZE = 128

tsfm = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
train_ds = TripletRETAIL(TRAIN_DIR,img_list,tsfm)
train_dl = DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle = False)

test_ds = RetailDataset(TEST_DIR,img_list_test,tsfm)
test_dl = DataLoader(test_ds, batch_size = BATCH_SIZE * 2, shuffle = False)

stocks_ds = RetailDataset(TRAIN_DIR,img_list,tsfm)
stocks_dl = DataLoader(stocks_ds, batch_size = BATCH_SIZE, shuffle = False)

In [ ]:
#!rm /content/TRAIN-embed/food_2282.jpg

In [ ]:
@torch.no_grad()
def validate(model,idx):
    outputs = None
    model.eval()
    for (i,batch) in enumerate(val_dl):
        batch = batch.to(DEVICE)
        index = i * BATCH_SIZE
        curr_batch_size = min(BATCH_SIZE, len(val_ds) - index)
        
        embed = model(batch).view(curr_batch_size,-1)
        embed = embed.cpu()
        print(embed.mean())
        if outputs is not None:
            outputs = torch.cat([outputs,embed],dim = 0)
        else:
            outputs = embed        
    return outputs

@torch.no_grad()
def test(model,ref_embeddings,test_embeddings,subset_sz = None):
    stocks_ds = None
    model.eval()
    preds = []
    correct_top1 = 0
    correct_top5 = 0
    correct_top10 = 0
    correct_top20 = 0

    if subset_sz is None:
        sz = len(test_ds)
    else:
        sz = subset_sz

    for i in tqdm(range(sz)):
        sample_test = test_embeddings[i]
        sample_test_bundle = torch.stack([sample_test for _ in range(len(ref_ds))]).to(GPU)
        distances = torch.linalg.norm(sample_test_bundle - ref_embeddings, dim = 1)
        #print(distances.device)
        # assert distances.device == GPU
        values,indices = distances.sort()

        top20 = [ref_ds.img_list[x][:-4] for x in indices[:20]]
        top10 = [ref_ds.img_list[x][:-4] for x in indices[:10]] 
        top5 = [ref_ds.img_list[x][:-4] for x in indices[:5]]
        top1 = ref_ds.img_list[indices[0]][:-4]
        
        preds.append(top5)
        #print(top5)
        #print(img2label[test_ds.img_list[i]])
        if img2label[test_ds.img_list[i]] in top5:
            correct_top5 += 1

        if img2label[test_ds.img_list[i]] in top10:
            correct_top10 += 1

        if img2label[test_ds.img_list[i]] in top20:
            correct_top20 += 1

        if img2label[test_ds.img_list[i]] == top1:
            correct_top1 += 1

    return preds,correct_top1/sz,correct_top5/sz,correct_top10/sz,correct_top20/sz

In [ ]:
preds ,acc1, acc5,acc10, acc20 = test(modelf,ref_embeddings,test_embeddings)
print("The accuracies are:\n Top 1 accuracy = {0}\n Top 5 accuracy = {1}\n Top 10 accuracy = {2}\n Top 20 accuracy = {3}".format(acc1,acc5,acc10,acc20))

In [ ]:
PATH = "/content/drive/MyDrive/ML_MODELS/resnet18_embed_gp_3.2k_ep6_full_tune.pt"
torch.save({"model" : model.state_dict(),
            "epochs" : 6},PATH)

In [ ]:
PATH = "/content/drive/MyDrive/ML_MODELS/resnet18_embed_gp_3.2k_ep6_OHNM.pt"
torch.save(best_ckpt,PATH)

In [ ]:
Top 1 accuracy = 0.7729672650475184
 Top 5 accuracy = 0.8595564941921858
 Top 10 accuracy = 0.8732840549102429
 Top 20 accuracy = 0.9028511087645196

In [ ]:
tSNE = TSNE(n_components=2)
ref_embed_2d = tSNE.fit_transform(ref_embeddings.cpu().numpy())

In [ ]:
plt.figure(figsize = (15,12))
plt.scatter(ref_embed_2d[:,0],ref_embed_2d[:,1])
plt.show()

In [ ]:
def plot_predictions(preds):
    for i in range(20):
        gtruth = cv2.imread(os.path.join(TEST_DIR,img_list_test[i]))
        img1 = cv2.imread(os.path.join(ROOT_DIR,preds[i][0] + ".jpg"))
        img2 = cv2.imread(os.path.join(ROOT_DIR,preds[i][1] + ".jpg"))
        img3 = cv2.imread(os.path.join(ROOT_DIR,preds[i][2] + ".jpg"))

        gtruth = cv2.cvtColor(gtruth,cv2.COLOR_BGR2RGB)
        img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
        img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2RGB)
        img3 = cv2.cvtColor(img3,cv2.COLOR_BGR2RGB)

        fig, axs = plt.subplots(1,4,figsize = (14,6))
        axs[0].imshow(gtruth)
        axs[0].set_title("Groud Truth")

        axs[1].imshow(img1)
        axs[1].set_title("Top-1")
        
        axs[2].imshow(img2)
        axs[2].set_title("Top-2")
        axs[3].imshow(img3)
        axs[3].set_title("Top-3")

In [ ]:
plot_predictions(preds)

In [ ]:
outputs = validate(model,0)
print(outputs.shape)

In [ ]:
print(outputs[18].mean())

In [ ]:
print(len(stocks_ds))

In [ ]:
#model.to(DEVICE)
model.to(torch.device('cpu'))
#del model

In [ ]:
anchor_image = torch.permute(val_ds[0],[1,2,0]).numpy() * 255
anchor_image = gauss_blur(anchor_image) 
anchor_image = randomCrop(anchor_image, 0.8, 0.8)
anchor_image = change_brightness_saturation(anchor_image, bright_f=40, sat_f=0.8)
anchor_image = cv2.resize(anchor_image, (224, 224))
anchor_image = torch.from_numpy(anchor_image).float()
anchor_image = torch.permute(anchor_image,[2,0,1])
anchor_image = val_ds.transform(anchor_image) / 255.0
with torch.no_grad():
    sample = anchor_image
    sample = torch.stack([sample for i in range(len(val_ds))]).to(DEVICE)
    ref_embed = model(sample).view(len(val_ds),-1)
    print(ref_embed.shape)

In [ ]:
ref_embed = ref_embed.to(torch.device('cpu'))
print(ref_embed.device)

In [ ]:
#distances = distance(outputs,ref_embed)
distances = torch.linalg.norm(outputs - ref_embed, dim = 1)
values,indices = distances.sort()
print(values[:11])
print(indices[:11])

In [ ]:
print(distances[:7])

In [ ]:
plt.plot(distances)
plt.show()

In [ ]:
print(distances)

In [ ]:
t = torch.stack([val_ds[0] for _ in range(10)])
print(t.shape)

In [ ]:
train_embeddings_tl, train_labels_tl = extract_embeddings(train_loader, model)
plot_embeddings(train_embeddings_tl, train_labels_tl)
val_embeddings_tl, val_labels_tl = extract_embeddings(test_loader, model)
plot_embeddings(val_embeddings_tl, val_labels_tl)

# Online pair/triplet selection - negative mining
There are couple of problems with siamese and triplet networks.
1. The number of possible pairs/triplets grows **quadratically/cubically** with the number of examples. It's infeasible to process them all
2. We generate pairs/triplets randomly. As the training continues, more and more pairs/triplets are easy to deal with (their loss value is very small or even 0), preventing the network from training. We need to provide the network with **hard examples**.
3. Each image that is fed to the network is used only for computation of contrastive/triplet loss for only one pair/triplet. The computation is somewhat wasted; once the embedding is computed, it could be reused for many pairs/triplets.

To deal with that efficiently, we'll feed a network with standard mini-batches as we did for classification. The loss function will be responsible for selection of hard pairs and triplets within mini-batch. In these case, if we feed the network with 16 images per 10 classes, we can process up to $159*160/2 = 12720$ pairs and $10*16*15/2*(9*16) = 172800$ triplets, compared to 80 pairs and 53 triplets in previous implementation.

We can find some strategies on how to select triplets in [2] and [3] *Alexander Hermans, Lucas Beyer, Bastian Leibe, [In Defense of the Triplet Loss for Person Re-Identification](https://arxiv.org/pdf/1703.07737), 2017*

## Online pair selection
## Steps
1. Create **BalancedBatchSampler** - samples $N$ classes and $M$ samples *datasets.py*
2. Create data loaders with the batch sampler
3. Define **embedding** *(mapping)* network $f(x)$ - **EmbeddingNet** from *networks.py*
4. Define a **PairSelector** that takes embeddings and original labels and returns valid pairs within a minibatch
5. Define **OnlineContrastiveLoss** that will use a *PairSelector* and compute *ContrastiveLoss* on such pairs
6. Train the network!

In [ ]:
from datasets import BalancedBatchSampler

# We'll create mini batches by sampling labels that will be present in the mini batch and number of examples from each class
train_batch_sampler = BalancedBatchSampler(train_dataset.train_labels, n_classes=10, n_samples=25)
test_batch_sampler = BalancedBatchSampler(test_dataset.test_labels, n_classes=10, n_samples=25)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
online_train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_batch_sampler, **kwargs)
online_test_loader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_batch_sampler, **kwargs)

# Set up the network and training parameters
from networks import EmbeddingNet
from losses import OnlineContrastiveLoss
from utils import AllPositivePairSelector, HardNegativePairSelector # Strategies for selecting pairs within a minibatch

margin = 1.
embedding_net = EmbeddingNet()
model = embedding_net
if cuda:
    model.cuda()
loss_fn = OnlineContrastiveLoss(margin, HardNegativePairSelector())
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 250

In [ ]:
all_embeddings = fit(online_train_loader, online_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

In [ ]:
train_embeddings_ocl, train_labels_ocl = extract_embeddings(train_loader, model)
plot_embeddings(train_embeddings_ocl, train_labels_ocl)
val_embeddings_ocl, val_labels_ocl = extract_embeddings(test_loader, model)
plot_embeddings(val_embeddings_ocl, val_labels_ocl)

## Online triplet selection
## Steps
1. Create **BalancedBatchSampler** - samples $N$ classes and $M$ samples *datasets.py*
2. Create data loaders with the batch sampler
3. Define **embedding** *(mapping)* network $f(x)$ - **EmbeddingNet** from *networks.py*
4. Define a **TripletSelector** that takes embeddings and original labels and returns valid triplets within a minibatch
5. Define **OnlineTripletLoss** that will use a *TripletSelector* and compute *TripletLoss* on such pairs
6. Train the network!

In [ ]:
from datasets import BalancedBatchSampler

# We'll create mini batches by sampling labels that will be present in the mini batch and number of examples from each class
train_batch_sampler = BalancedBatchSampler(train_dataset.train_labels, n_classes=10, n_samples=25)
test_batch_sampler = BalancedBatchSampler(test_dataset.test_labels, n_classes=10, n_samples=25)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
online_train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_batch_sampler, **kwargs)
online_test_loader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_batch_sampler, **kwargs)

# Set up the network and training parameters
from networks import EmbeddingNet
from losses import OnlineTripletLoss
from utils import AllTripletSelector,HardestNegativeTripletSelector, RandomNegativeTripletSelector, SemihardNegativeTripletSelector # Strategies for selecting triplets within a minibatch
from metrics import AverageNonzeroTripletsMetric

margin = 1.
embedding_net = EmbeddingNet()
model = embedding_net
if cuda:
    model.cuda()
loss_fn = OnlineTripletLoss(margin, RandomNegativeTripletSelector(margin))
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 150

In [ ]:
fit(online_train_loader, online_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[AverageNonzeroTripletsMetric()])

In [ ]:
train_embeddings_otl, train_labels_otl = extract_embeddings(train_loader, model)
plot_embeddings(train_embeddings_otl, train_labels_otl)
val_embeddings_otl, val_labels_otl = extract_embeddings(test_loader, model)
plot_embeddings(val_embeddings_otl, val_labels_otl)

In [ ]:
display_emb_online, display_emb, display_label_online, display_label = train_embeddings_ocl, train_embeddings_cl, train_labels_ocl, train_labels_cl
# display_emb_online, display_emb, display_label_online, display_label = val_embeddings_ocl, val_embeddings_cl, val_labels_ocl, val_labels_cl

x_lim = (np.min(display_emb_online[:,0]), np.max(display_emb_online[:,0]))
y_lim = (np.min(display_emb_online[:,1]), np.max(display_emb_online[:,1]))
x_lim = (min(x_lim[0], np.min(display_emb[:,0])), max(x_lim[1], np.max(display_emb[:,0])))
y_lim = (min(y_lim[0], np.min(display_emb[:,1])), max(y_lim[1], np.max(display_emb[:,1])))

plot_embeddings(display_emb, display_label, x_lim, y_lim)
plot_embeddings(display_emb_online, display_label_online, x_lim, y_lim)

In [ ]:
display_emb_online, display_emb, display_label_online, display_label = train_embeddings_otl, train_embeddings_tl, train_labels_otl, train_labels_tl
# display_emb_online, display_emb, display_label_online, display_label = val_embeddings_otl, val_embeddings_tl, val_labels_otl, val_labels_tl
x_lim = (np.min(display_emb_online[:,0]), np.max(display_emb_online[:,0]))
y_lim = (np.min(display_emb_online[:,1]), np.max(display_emb_online[:,1]))
x_lim = (min(x_lim[0], np.min(display_emb[:,0])), max(x_lim[1], np.max(display_emb[:,0])))
y_lim = (min(y_lim[0], np.min(display_emb[:,1])), max(y_lim[1], np.max(display_emb[:,1])))
plot_embeddings(display_emb, display_label, x_lim, y_lim)
plot_embeddings(display_emb_online, display_label_online, x_lim, y_lim)